In [1]:
# Main file
# Use case: Pilot test, "integration test"

# === STEP 0: Imports
print("# === STEP 0: Imports: Starting ...")
from modules.dataset_generator.helpers.configuration_loader import ConfigurationLoader as DSConfigLoader
from modules.dataset_generator.factories.data_io_factory import DataIOFactory
from modules.dataset_generator.factories.feature_processor_factory import FeatureProcessorFactory
from modules.dataset_generator.factories.join_factory import JoinFactory
from modules.dataset_generator.factories.strategy_factory import StrategyFactory
from modules.dataset_generator.dataset_generator import DatasetGenerator
from modules.processor.helpers.configuration_loader import ConfigurationLoader as PConfigLoader
from modules.processor.factories.split_strategy_factory import SplitStrategyFactory
from modules.processor.processor import Processor
from modules.model_manager.trainer.trainer import Trainer
from modules.model_manager.predictor.predictor import Predictor
from modules.model_manager.factories.model_factory import ModelFactory
from modules.model_manager.helpers.configuration_loader import ConfigurationLoader as MMConfigLoader
from modules.model_manager.model_manager import ModelManager
print("# === STEP 0: Imports: Complete")

# === STEP 0: Imports: Starting ...


2025-01-13 08:53:45.107319: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# === STEP 0: Imports: Complete


In [3]:
# === STEP 1: Dependency Instantiations And Global Variable Declarations
print("# === STEP 1: Dependency Instantiations And Global Variable Declarations: Starting ...")
## === DATASET GEN
yaml_path = '/Users/joaquinuriarte/Documents/GitHub/sports-betting/configs/model_v0.yaml'
ds_configuration_loader = DSConfigLoader()
data_factory, feature_processor_factory, join_factory, strategy_factory = DataIOFactory(), FeatureProcessorFactory(), JoinFactory(), StrategyFactory()
## === PROCESSOR
p_configuration_loader = PConfigLoader()
split_strategy_factory = SplitStrategyFactory()
## === MODEL MANAGER
checkpoint = '/Users/joaquinuriarte/Documents/GitHub/sports-betting/models'
trainer = Trainer(checkpoint)
predictor = Predictor()
model_factory = ModelFactory()
mm_configuration_loader = MMConfigLoader()
print("# === STEP 1: Dependency Instantiations And Global Variable Declarations: Complete")

# === STEP 1: Dependency Instantiations And Global Variable Declarations: Starting ...
# === STEP 1: Dependency Instantiations And Global Variable Declarations: Complete


In [4]:
# === STEP 2: DATASET GEN
print("# === STEP 2: DATASET GEN: Starting ...")
dataset_generator = DatasetGenerator(yaml_path, ds_configuration_loader, data_factory, feature_processor_factory, join_factory, strategy_factory)

# === STEP 2: DATASET GEN: Starting ...


In [ ]:
processed_dataset = dataset_generator.generate()
print("# === STEP 2: DATASET GEN: Complete")

In [5]:
# File path to save and load the processed dataset
processed_dataset_path = "/Users/joaquinuriarte/Documents/GitHub/sports-betting/processed_datasets/processed_dataset.pkl"

In [ ]:
## Use to save processed dataset to disk ## 
import pickle
import os

with open(processed_dataset_path, "wb") as f:
        pickle.dump(processed_dataset, f)
        print("# === STEP 2: DATASET GEN: Saved to file")

In [6]:
## Use to load processed dataset if computed already ## 
import pickle
import os

print("# === STEP 2: DATASET GEN: Loading from saved file")
with open(processed_dataset_path, "rb") as f:
    processed_dataset = pickle.load(f)

# === STEP 2: DATASET GEN: Loading from saved file


In [7]:
processed_dataset.features

,A_player_1_MIN,A_player_1_PTS,A_player_1_AST,A_player_1_TO,A_player_1_PLUS_MINUS,A_player_1_OREB,A_player_1_DREB,A_player_1_PF,A_player_1_FG3_PCT,A_player_1_FG_PCT,...,B_player_8_AST,B_player_8_TO,B_player_8_PLUS_MINUS,B_player_8_OREB,B_player_8_DREB,B_player_8_PF,B_player_8_FG3_PCT,B_player_8_FG_PCT,B_player_8_FT_PCT,Team_A_Wins
GAME_ID,,,,,,,,,,,,,,,,,,,,,
11400001,41.266667,31.0,5.0,1.0,-16.0,1.0,9.0,1.0,0.333,0.476,...,0.0,0.0,-11.0,0.0,0.0,4.0,0.0000,0.6670,0.0,0
11400002,45.083333,18.0,6.0,2.0,2.0,0.0,2.0,2.0,0.500,0.417,...,4.0,2.0,5.0,0.0,0.0,2.0,0.0000,0.4000,0.0,1
11400004,43.200000,6.0,7.0,4.0,-1.0,2.0,16.0,1.0,0.000,0.375,...,0.0,0.0,2.0,0.0,0.0,1.0,0.0000,0.0000,0.0,0
11400005,36.600000,12.0,2.0,1.0,9.0,1.0,7.0,3.0,0.000,0.545,...,1.0,0.0,6.0,0.0,5.0,4.0,0.5000,0.6670,1.0,1
11400006,36.866667,18.0,3.0,5.0,-17.0,2.0,3.0,4.0,0.429,0.389,...,0.0,0.0,13.0,0.0,3.0,4.0,0.0000,0.6000,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52100111,32.566667,22.0,0.0,2.0,18.0,0.0,2.0,1.0,1.000,0.538,...,0.0,0.0,-4.0,0.0,0.0,0.0,0.0000,0.0000,0.0,1
52100121,21.316667,13.0,2.0,0.0,-3.0,3.0,0.0,4.0,0.000,0.714,...,2.5,0.5,14.5,0.0,3.0,2.0,0.4165,0.4285,0.5,1
52100131,32.316667,19.0,6.0,3.0,-13.0,3.0,2.0,2.0,0.000,0.533,...,1.0,0.0,-1.0,2.0,2.0,0.0,0.0000,0.5000,1.0,1


In [8]:
# === STEP 3: PROCESSOR
print("# === STEP 3: PROCESSOR: Starting ...")
processor = Processor(yaml_path, p_configuration_loader, processed_dataset, split_strategy_factory)

# === STEP 3: PROCESSOR: Starting ...


In [9]:
train_dataset, validation_dataset, test_dataset = processor.generate()
print("# === STEP 3: PROCESSOR: Complete")

# === STEP 3: PROCESSOR: Complete


In [16]:
len(test_dataset.examples)

25141

In [13]:
print("# === STEP 3.5: Assessing dataset inbalance: Starting...")
from collections import Counter
# Function to calculate dataset balance
def calculate_balance(dataset, name):
    # Extract "Team_A_Wins" labels
    y_labels = [example.features["Team_A_Wins"][0] for example in dataset.examples]

    # Count occurrences of each label
    label_counts = Counter(y_labels)
    num_1s = label_counts[1]
    num_0s = label_counts[0]

    # Calculate proportions
    total_samples = len(y_labels)
    proportion_1s = num_1s / total_samples * 100
    proportion_0s = num_0s / total_samples * 100

    # Display results
    print(f"Number of 1s (Team A wins) in {name}: {num_1s} ({proportion_1s:.2f}%)")
    print(f"Number of 0s (Team A does not win) in {name}: {num_0s} ({proportion_0s:.2f}%)")

# Assuming your dataset is loaded into memory as `my_dataset`
calculate_balance(train_dataset, "train")
calculate_balance(validation_dataset, "val")
calculate_balance(test_dataset, "test")

print("# === STEP 3.5: Assessing dataset inbalance: Complete")

# === STEP 3.5: Assessing dataset inbalance: Starting...
Number of 1s (Team A wins) in train: 112 (62.92%)
Number of 0s (Team A does not win) in train: 66 (37.08%)
Number of 1s (Team A wins) in val: 64 (50.79%)
Number of 0s (Team A does not win) in val: 62 (49.21%)
Number of 1s (Team A wins) in test: 14821 (58.95%)
Number of 0s (Team A does not win) in test: 10320 (41.05%)
# === STEP 3.5: Assessing dataset inbalance: Complete


In [10]:
# === STEP 4: MODEL MANAGER
print("# === STEP 4: MODEL MANAGER: Starting ...")
model_manager = ModelManager(trainer, predictor, model_factory, mm_configuration_loader)
models = model_manager.create_models([yaml_path])

# === STEP 4: MODEL MANAGER: Starting ...


In [10]:
train_dataset.examples[0].features
len(train_dataset.examples[0].features) # should be 177

177

In [11]:
# If no val dataset, code requires to push None instead
val_dataset = validation_dataset if validation_dataset is not None else None

model_manager.train(models, [(train_dataset, val_dataset)], save_after_training=True)
print("# === STEP 4: MODEL MANAGER: Complete")

INFO:root:Training model '8997782287ac95361ce10ecee1af1840' for 20 epochs with batch size 32.
INFO:root:Model '8997782287ac95361ce10ecee1af1840': Starting epoch 1/20.
/Users/joaquinuriarte/Documents/GitHub/sports-betting/sports-betting/lib/python3.8/site-packages/keras/src/backend.py:5805: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


6/6 [==============================] - 1s 75ms/step - loss: 2.3138 - accuracy: 0.5955 - val_loss: 3.1550 - val_accuracy: 0.4603


INFO:root:Model '8997782287ac95361ce10ecee1af1840': Starting epoch 2/20.


6/6 [==============================] - 0s 37ms/step - loss: 2.0116 - accuracy: 0.5449 - val_loss: 2.0170 - val_accuracy: 0.5635


INFO:root:Model '8997782287ac95361ce10ecee1af1840': Starting epoch 3/20.


...

6/6 [==============================] - 0s 27ms/step - loss: 0.0567 - accuracy: 1.0000 - val_loss: 1.4785 - val_accuracy: 0.5635


INFO:root:Model '8997782287ac95361ce10ecee1af1840': Starting epoch 20/20.


6/6 [==============================] - 0s 25ms/step - loss: 0.0459 - accuracy: 1.0000 - val_loss: 1.5144 - val_accuracy: 0.5556


INFO:root:Model '8997782287ac95361ce10ecee1af1840': Finished training.


Model saved successfully at: models/8997782287ac95361ce10ecee1af1840/model_weights_8997782287ac95361ce10ecee1af1840.pth
# === STEP 4: MODEL MANAGER: Complete


In [12]:
predictions = []
for model in models:
    print(model)
    predictions.append(model.predict(test_dataset.examples, return_target_labels=True))

In [13]:
for df in predictions:
    correct_predictions = (df['predictions'] == df['target_label']).sum()
    total_predictions = len(df)
    accuracy = correct_predictions / total_predictions
    print(f"Accuracy on test dataset: {accuracy}")

Accuracy on test dataset: 0.5895151346406269


In [15]:
%tensorboard --logdir logs/fit

UsageError: Line magic function `%tensorboard` not found.
